# Sprint8 機械学習スクラッチ アンサンブル学習

## 1.このSprintについて

### Sprintの目的
- アンサンブル学習について理解する
### どのように学ぶか
スクラッチでアンサンブル学習の各種手法を実装していきます。

## 2.アンサンブル学習
3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。

- ブレンディング
- バギング
- スタッキング

### 小さなデータセットの用意
以前も利用した回帰のデータセットを用意します。

[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。

train.csvを学習用（train）8割、検証用（val）2割に分割してください。

### scikit-learn
単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。

[sklearn.linear_model.LinearRegression — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

[sklearn.svm.SVR — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)

[sklearn.tree.DecisionTreeRegressor — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import random
import math

In [2]:
df = pd.read_csv("train.csv")
X = df[["GrLivArea", "YearBuilt"]].values
y = df["SalePrice"].values

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [4]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_y_pred = lr.predict(X_test)
print("線形回帰 MSE : {}".format(mean_squared_error(y_test, lr_y_pred)))

線形回帰 MSE : 2495554898.6683216


In [5]:
svm = SVR()
svm.fit(X_train, y_train)
svm_y_pred = svm.predict(X_test)
print("SVM MSE : {}".format(mean_squared_error(y_test, svm_y_pred)))

SVM MSE : 7861854841.842987


C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [6]:
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
tree_y_pred = tree.predict(X_test)
print("決定木 MSE : {}".format(mean_squared_error(y_test, tree_y_pred)))

決定木 MSE : 2204076821.5252094


## 3.ブレンディング
＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿
### 【問題1】ブレンディングのスクラッチ実装
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

### ブレンディングとは
ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。

- 手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
- ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
- 入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。

回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。

#### 《補足》

分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。

[sklearn.ensemble.VotingClassifier — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html)

In [7]:
def blend(X_train, X_test, y_train, model1, model2):
    model1.fit(X_train, y_train)
    model1_pred = model1.predict(X_test)
    
    model2.fit(X_train, y_train)
    model2_pred = model2.predict(X_test)
    y_pred = (model1_pred + model2_pred) / 2
    return y_pred

In [8]:
# 線形回帰と決定木の予測結果の平均を予測値とする
y_pred = blend(X_train, X_test, y_train, lr, tree)
print("   線形回帰     MSE : {}".format(mean_squared_error(y_test, lr_y_pred)))
print("      SVM       MSE : {}".format(mean_squared_error(y_test, svm_y_pred)))
print("    決定木      MSE : {}".format(mean_squared_error(y_test, tree_y_pred)))
print("線形回帰+決定木 MSE : {}".format(mean_squared_error(y_test, y_pred)))

   線形回帰     MSE : 2495554898.6683216
      SVM       MSE : 7861854841.842987
    決定木      MSE : 2204076821.5252094
線形回帰+決定木 MSE : 1861935310.537359


In [9]:
# SVMと決定木の予測結果の平均を予測値とする
y_pred = blend(X_train, X_test, y_train, svm, tree)
print("線形回帰   MSE : {}".format(mean_squared_error(y_test, lr_y_pred)))
print("SVM        MSE : {}".format(mean_squared_error(y_test, svm_y_pred)))
print("決定木     MSE : {}".format(mean_squared_error(y_test, tree_y_pred)))
print("SVM+決定木 MSE : {}".format(mean_squared_error(y_test, y_pred)))

線形回帰   MSE : 2495554898.6683216
SVM        MSE : 7861854841.842987
決定木     MSE : 2204076821.5252094
SVM+決定木 MSE : 3390320870.5819483


C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [10]:
# SVMの線形カーネルと多項式カーネルのモデルの予測の平均
model1 = SVR(gamma='scale', kernel='linear')
model2 = SVR(gamma='scale') 
y_pred = blend(X_train, X_test, y_train, model1, model2)
print("   線形回帰     MSE : {}".format(mean_squared_error(y_test, lr_y_pred)))
print("      SVM       MSE : {}".format(mean_squared_error(y_test, svm_y_pred)))
print("    決定木      MSE : {}".format(mean_squared_error(y_test, tree_y_pred)))
print("SVM 線形+多項式 MSE : {}".format(mean_squared_error(y_test, y_pred)))

   線形回帰     MSE : 2495554898.6683216
      SVM       MSE : 7861854841.842987
    決定木      MSE : 2204076821.5252094
SVM 線形+多項式 MSE : 4450951532.463163


## 4.バギング
＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿
### 【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

### バギングとは
バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。

[sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。

推定結果の平均をとる部分はブースティングと同様の実装になります。

In [11]:
def bagging(X_train, X_test, y_train, model, n=2):
    y_pred = np.zeros(X_test.shape[0])
    for i in range(n):
        X_divided = train_test_split(X_train, random_state=random.randint(0, i))[0]
        y_divided = train_test_split(X_train, random_state=random.randint(0, i))[0]
        model.fit(X_train, y_train)
        y_pred += model.predict(X_test)
    y_pred = y_pred / n
    return y_pred

y_pred = bagging(X_train, X_test, y_train, tree, n=100)

print("    決定木      MSE : {}".format(mean_squared_error(y_test, tree_y_pred)))
print("bagging 決定木  MSE : {}".format(mean_squared_error(y_test, y_pred)))

    決定木      MSE : 2204076821.5252094
bagging 決定木  MSE : 2159302214.8116994


## 5.スタッキング
＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿＿
### 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

### スタッキングとは
スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは 
$K_0 = 3, M_0 = 2$ 程度にします。

#### 《学習時》

#### （ステージ 0 ）

- 学習データを $K_0$ 個に分割する。
- 分割した内の $(K_0 − 1)$ 個をまとめて学習用データ、残り 1 個を推定用データとする組み合わせが $K_0$ 個作れる。
- あるモデルのインスタンスを $K_0$ 個用意し、異なる学習用データを使い学習する。
- それぞれの学習済みモデルに対して、使っていない残り 1 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
- さらに、異なるモデルのインスタンスも $K_0$ 個用意し、同様のことを行う。モデルが $M_0$ 個あれば、$M_0$ 個のブレンドデータが得られる。

#### （ステージ n ）

- ステージ $n −1 $ のブレンドデータを$M_n − 1$ 次元の特徴量を持つ学習用データと考え、$K_n$ 個に分割する。以下同様である。

#### （ステージ N ）＊最後のステージ

- ステージ$N − 1$ の$M_{N-1}$ 個のブレンドデータを$M_{N−1}$ 次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

#### 《推定時》

#### （ステージ 0 ）

- テストデータを $K_0×M_0$ 個の学習済みモデルに入力し、$K_0×M_0$ 個の推定値を得る。これを$K_0$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

#### （ステージ n ）

- ステージ $n−1$ で得たブレンドテストを$K_n×M_n$ 個の学習済みモデルに入力し、$K_n×M_n$ 個の推定値を得る。これを_\K_n$ の軸で平均値を求め $M_0 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

#### （ステージ N ）＊最後のステージ

- ステージ$N−1$ で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

In [12]:
class Stacking():
    def __init__(self, split_n=3, model_n=2):
        self.split_n = split_n
        self.model_n = model_n

    def fit(self, X_train, y_train, X_test, y_test, models):
        # K個に分割するdividerを作る
        divider = np.zeros(self.split_n)
        vol = X_train.shape[0]
        num = self.split_n
        for i in range(self.split_n):
            divider[i] = math.ceil(vol/num)
            num -= 1
            vol = vol-divider[i]
        
        self.divider = divider.astype(int)
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.models = models
        print(self.divider)
    
    def predict(self, X_test):
        for m in range(self.model_n):
            divide_point = 0
            for n in range(self.split_n):
                idx = np.zeros(X_train.shape[0], dtype=bool)
                idx[divide_point:divide_point+self.divider[n]]= True
                self.X_test_divided = X_train[idx, :]
                self.X_train_divided = X_train[~idx, :]
                self.y_test_divided = y_train[idx]
                self.y_train_divided = y_train[~idx]                    
                
                models[m].fit(self.X_train_divided, self.y_train_divided)
                if n == 0:
                    blend = models[m].predict(self.X_test_divided)
                    pred_data = models[m].predict(X_test)
                else:
                    blend = np.r_[blend, models[m].predict(self.X_test_divided)]
                    pred_data = np.c_[pred_data, models[m].predict(X_test)]
            
                divide_point += self.divider[n]
            if m ==0:
                blend_data =blend.reshape(-1, 1)
                blend_pred_data = np.mean(pred_data, axis=1)
            else:
                blend_data = np.c_[blend_data, blend.reshape(-1, 1) ]
                blend_pred_data = np.c_[blend_pred_data, np.mean(pred_data, axis=1)]
        
        models[0].fit(blend_data, y_train)
        y_pred = models[0].predict(blend_pred_data)                               
        return y_pred

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
models = [LinearRegression(), DecisionTreeRegressor()]

stacking = Stacking()
stacking.fit(X_train, y_train, X_test, y_test, models)
y_pred = stacking.predict(X_test)

print("線形回帰 MSE : {}".format(mean_squared_error(y_test, lr_y_pred)))
print("SVM      MSE : {}".format(mean_squared_error(y_test, svm_y_pred)))
print("決定木   MSE : {}".format(mean_squared_error(y_test, tree_y_pred)))
print("Stacking MSE : {}".format(mean_squared_error(y_test, y_pred)))


[390 389 389]
線形回帰 MSE : 2495554898.6683216
SVM      MSE : 7861854841.842987
決定木   MSE : 2204076821.5252094
Stacking MSE : 1932374160.447718
